# Retriever and Chain with Langchain

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("*******")
docs = loader.load()
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
# doc_split = text_splitter.split_documents(docs)[:5]
doc_split = text_splitter.split_documents(docs)
doc_split

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma, FAISS

db = Chroma.from_documents(doc_split[:10], OllamaEmbeddings(model="llama3.2"))
db

In [ ]:
query = "print the first line of the document"
result = db.similarity_search(query)
result[0].page_content

In [ ]:
# from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")
llm

In [11]:
# Design ChatPromt Template
from langchain_core.prompts import ChatPromptTemplate
promt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}""")

# Chain Introduction

In [14]:
# Create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, promt)

In [ ]:
# Retriever

retriever = db.as_retriever()
retriever

In [ ]:
# Retriever Chain

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input":"Research Assistant - ML/DL Engineer\nDeutsches Forschungszentrum für Künstliche Intelligenz"})

In [ ]:
response['answer']